# Enabling Real-Time Feedback

A major benefit of HRNN-based models over, say, traditional collaborative filtering systems is that our recommendations can respond to a user's actions in the session.

Our CloudFormation stack has already laid down the infrastructure for the website to report click (view) events, so in this notebook we review the actions in Amazon Personalize to enable real-time event tracking.

## An important note on accuracy

The public [UCSD datasets](https://nijianmo.github.io/amazon/index.html) used for this data comprise **reviews**, which are more sparse than purchase data and **a lot more sparse** than *page views* - which is how the models are being presented in the website.

Because of this, we need to bear in mind that even as we make more improvements to our solution, the recommendations might sometimes seem a little odd at first glance...

## Getting started

first up we'll load some variables that were stored by notebook 1, and create a Personalize API client as we did before:

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import boto3
from botocore import exceptions as botoexceptions

%store -r role_arn
%store -r dataset_group_arn
%store -r hrnn_campaign_arn

personalize = boto3.client("personalize")

## Creating the Event Tracker

We create an **Event Tracker** via the Personalize API, which will get us a **Tracking ID** with which live events can be recorded at runtime.

**Note that each Dataset Group can have only one Event Tracker**: We don't need to worry about separate tracking IDs to record events for different deployed campaigns in the same dataset group. Our website stack is set up to expect a single tracking ID only.

In [ ]:
try:
    create_tracker_response = personalize.create_event_tracker(
        name=f"{os.environ['CF_STACK_NAME']}-tracker",
        datasetGroupArn=dataset_group_arn
    )

    event_tracker_arn = create_tracker_response["eventTrackerArn"]
    tracking_id = create_tracker_response["trackingId"]
    print(f"Created tracker\nARN: {event_tracker_arn}\nTracking ID: {tracking_id}")

except botoexceptions.ClientError as err:
    # If the tracker already exists, we'll just use the existing:
    if err.response["Error"]["Code"] == "ResourceAlreadyExistsException":
        print("Event Tracker already exists - scraping ARN from error message")
        msg = err.response["Error"]["Message"]
        event_tracker_arn = msg[msg.index("arn:aws:personalize"):].partition(" ")[0]
        description = personalize.describe_event_tracker(eventTrackerArn=event_tracker_arn)
        print(description)
        tracking_id = description["eventTracker"]["trackingId"]
        print(f"Existing tracker\nARN: {event_tracker_arn}\nTracking ID: {tracking_id}")
    else:  # Some other problem
        raise err

## Configuring the Tracking ID in the website

The website is already set up to:

- Collect click (view) events from the front end and publish them to a [Kinesis Stream](https://aws.amazon.com/kinesis/data-streams/)
- Process incoming data on the Kinesis stream with the `xxx-PostClickEvent` **Lambda Function**, which will
- Record the events to the configured (via environment variable) Personalize Tracking ID.

So once our tracking ID is set up, it really is as straightforward as updating an environment variable on our Lambda function, to start pushing live events through:

In [ ]:
from util.lambdafn import update_lambda_envvar

update_lambda_envvar(os.environ["LAMBDA_POSTCLICK_ARN"], "TRACKING_ID", tracking_id)

## That's pretty much it!

The best way to see the results in action is to **click around the website as a logged-in user whose ID is present in the dataset**. If your user sees different homepage recommendations to the default/anonymous (an indicator they were recognised), then you should start to see recommendations change as you browse around the items.

Just like requesting recommendations uses the [personalize-runtime SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize-runtime.html) (slightly different from the [personalize SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html) for managing models), event logging uses the [personalize-events SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize-events.html).

The flow is simple enough, with just one [`put_events()`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize-events.html#PersonalizeEvents.Client.put_events) method to look out for. See the implementation of the `{stackname}-PostClickEvent` function from the [Lambda Console](https://console.aws.amazon.com/lambda/home?#/functions) for full details!

In [ ]:
# As before, you could de-activate tracking again on the website like this:
# update_lambda_envvar(
#     os.environ["LAMBDA_POSTCLICK_ARN"],
#     "TRACKING_ID",
#     ""
# )

## (Optional): Simulating user behaviour

It's also possible for us to use the personalize-events SDK **here in the notebook** to see how a user's recommendations will change after events are recorded.

We'll start by setting up a session dictionary (so repeated experiments with the same user carry over) and defining the function to send dummy click events to Personalize:

In [ ]:
from collections import defaultdict
import json
import time
import uuid

personalize_events = boto3.client("personalize-events")
session_dict = defaultdict(uuid.uuid1) # Dict from user ID to first-fetch-randomized session ID

def send_interaction(user_id, item_id):
    """Send a user-item interaction to Amazon Personalize"""
    session_id = session_dict[user_id]

    return personalize_events.put_events(
        trackingId=tracking_id,  # From earlier
        userId=user_id,
        sessionId=str(session_id),
        eventList=[{  # It's actually a batch operation, but we're sending one at a time
            "sentAt": int(time.time()),
            "eventType": "EVENT_TYPE",
            "properties": json.dumps({
                "itemId": str(item_id),
            }),
        }],
    )

Next, we pick a user ID to play with:

In [ ]:
import random
from util import dataformat

n_users = 0
sample_user = None

for user in dataformat.data_folder_reader("data/raw/users"):
    # Reservoir sampling R-algorithm (simple, non-optimal) with k=1:
    n_users += 1
    if random.randint(1, n_users) <= 1:
        sample_user = user

user_id = dataformat.get_user_id(sample_user)
print(f"Testing with user ID {user_id} ({user.get('firstName', '???')} {user.get('lastName', '???')})")

Then we'll build up a dictionary from item ID to readable title (so we can make any sense of the results), and pick a list of one or more items to interact with:

In [ ]:
# Configure number of interactions here:
n_interactions = 3

import pandas as pd

item_titles = {}
n_items = 0
interaction_items = []

for item in dataformat.data_folder_reader("data/raw/items"):
    n_items += 1
    item_titles[dataformat.get_item_id(item)] = dataformat.get_item_title(item)

    # Reservoir sampling R-algorithm (simple, non-optimal) with k=1:
    if n_items <= n_interactions:
        # Fill up the reservoir first:
        interaction_items.append(dataformat.get_item_id(item))
    else:
        # Randomly resample for remainder of data:
        i = random.randint(n_interactions, n_items)
        if i <= n_interactions:
            interaction_items[i - 1] = dataformat.get_item_id(item)

print(f"\nGot {n_items} items total ({len(item_titles)} unique IDs)")
print("Interacting with items:")
pd.concat(
    [
        pd.Series(interaction_items, name="Item ID"),
        pd.Series([item_titles[iid] for iid in interaction_items], name="Title")
    ],
    axis=1
)

We now have everything ready for our simulation:

* The user ID to impersonate
* The list of item IDs to interact with, in order
* The Tracking ID for our dataset group, obtained earlier
* The campaign ARN for our target recommendation model (the basic one from notebook 1)
* A session ID randomly generated for us once for each user we test with

All that's left is to simulate the interactions, and visualize the recommendations before and after:

In [ ]:
# TODO: Factor user and item loading logic, and item title display logic, into utilities shared with 1.
import pandas as pd

personalize_runtime = boto3.client("personalize-runtime")

# Fetch the initial recommendations:
initial_recs = [
    item_titles[item["itemId"]]
    for item in personalize_runtime.get_recommendations(
        campaignArn=hrnn_campaign_arn,
        userId=str(user_id),
    )["itemList"]
]

# Send the interactions:
print("Simulating interactions...")
for item_id in interaction_items:
    send_interaction(user_id, item_id)
    time.sleep(3)  # Include a little bit of delay to be semi-realistic at least

# Fetch the final recommendations:
print("Fetching results...")
final_recs = [
    item_titles[item["itemId"]]
    for item in personalize_runtime.get_recommendations(
        campaignArn=hrnn_campaign_arn,
        userId=str(user_id),
    )["itemList"]
]

pd.concat(
    [
        pd.Series(initial_recs, name="Initial Recommendations"),
        pd.Series(final_recs, name="Final Recommendations"),
    ],
    axis=1
).head(15)